In [3]:
import bqplot
import pandas as pd
import numpy as np
import ipywidgets
import traitlets

In [4]:
states = pd.read_csv("us-states.csv", parse_dates = ["date"])
counties = pd.read_csv("us-counties.csv", parse_dates = ["date"])

In [5]:
proj = bqplot.AlbersUSA()
mark = bqplot.Map(
    map_data = bqplot.topo_load("map_data/USStatesMap.json"),
    scales = {'projection': proj}
)
fig = bqplot.Figure(marks = [mark])
display(fig)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, marks=[Map(hovered_styles={'hovered_fill…

In [19]:
case_counts = states.groupby("fips")["cases"].max()
case_counts.index = case_counts.index.astype('int64')
case_counts = case_counts.to_dict()

proj = bqplot.AlbersUSA()

color_sc = bqplot.ColorScale(scheme = "viridis")
color_ax = bqplot.ColorAxis(scale = color_sc, label = 'Case Count')

mark = bqplot.Map(
    map_data = bqplot.topo_load("map_data/USStatesMap.json"),
    color = case_counts,
    scales = {'projection': proj, 'color': color_sc}
)

import ipywidgets
label = ipywidgets.Label()

def hover_over_state(thismark, name):
    label.value = "%s" % (name)

mark.on_hover(hover_over_state)

fig = bqplot.Figure(marks = [mark], axes =[color_ax])
ipywidgets.VBox([label, fig])

In [14]:
proj.translate = (600, 500)
proj.scale_factor = 100

In [23]:
illinois_cases = counties[counties["state"] == "Illinois"].groupby("fips")["cases"].max()
illinois_cases.index = illinois_cases.index.astype("int64")

county_cases_by_state = {}

for state in counties["state"].unique():
    state_cases = counties[counties["state"] == state].groupby("fips")["cases"].max()
    state_cases.index = state_cases.index.astype("int64")
    county_cases_by_state[state] = state_cases

In [22]:
proj = bqplot.AlbersUSA()

color_sc = bqplot.ColorScale(scheme = "viridis")
color_ax = bqplot.ColorAxis(scale = color_sc, label = 'Case Count')

mark = bqplot.Map(
    map_data = bqplot.topo_load("map_data/USStatesMap.json"),
    color = case_counts,
    scales = {'projection': proj, 'color': color_sc}
)
label = ipywidgets.Label()

def hover_over_state(mark, event):
    label.value = "%s" % (event['data'])

mark.on_hover(hover_over_state)
mark.hovered_styles = {'hovered_fill': 'none', 'hovered_stroke': 'white', 'hovered_stroke_width': 5.0}

fig_states = bqplot.Figure(marks = [mark], axes =[color_ax])

proj = bqplot.AlbersUSA()

color_sc = bqplot.ColorScale(scheme = "viridis")
color_ax = bqplot.ColorAxis(scale = color_sc, label = 'Case Count')

mark = bqplot.Map(
    map_data = bqplot.topo_load("map_data/USCountiesMap.json"),
    color = illinois_cases.to_dict(),
    colors = {'default_color': 'none'},
    scales = {'projection': proj, 'color': color_sc}
)
label = ipywidgets.Label()

fig_counties = bqplot.Figure(marks = [mark], axes =[color_ax])



def on_state_hover(mark, event):
    mark_counties.color = county_cases_by_state[event['data']['name']].to_dict()

mark_states.on_hover(on_state_hover)

display(ipywidgets.VBox([fig_states, fig_counties]))